# About this module

The module scrapes Eurovision data off of the main site which stores that data, which is https://eschome.net/index.html

The structure of the site is to provide the user with various slices of the data, which are presented in html tables.  No direct access to the underlying data is available.

At first glance, it seems like eschome.net is dynamically rendering the data in a way that will make it hard to scrape, but as you investigate the site, it turns out you can reverse-engineer the html POST operations which render the data pages, so even though the actual database calls are hidden in some php code, you can treat the resulting pages as static html.

This allows the data scraping to be done very efficiently using the pandas read_html() function, which locates tables on web pages and puts them into dataframes.

The data of interest for this presentation is:

* List of every Eurovision final (reference)

* List of years, countries that participated in the finals that year, the order in which they performed in the finals, and how they placed. (to allow analysis of how important it is which order you perform in)

* List of years, participant countries, and how they voted (to allow analysis of block voting, like "all the Baltic countries vote together")

__I've put a variable called "is_refresh" around all of the code to avoid needlessly re-scraping the site, which only changes once per year, and then, only to add.  Set is_refresh to True at the top of the code to re-scrape.__

## Get list of all the finals, and where they were hosted

In [ ]:
import requests
import pandas as pd
from io import StringIO as SIO

is_refresh = False

if is_refresh == True:

    # url generated by eschome.net when you click on "List of all Final Events" (no details)
    url = 'https://eschome.net/databaseoutput410.php'

    # get the full page text
    page = requests.post(url)

    # create a list of tables
    list_of_tables = pd.read_html(SIO(page.text), header = 0)

    # assign second table to dataframe and keep only the interesting columns
    df = list_of_tables[1]
    all_finals = df[['Year','Country','City','Location','Broadcaster','Date']]
    print("Imported " + str(len(all_finals)) + " finals records.")
    all_finals.to_csv('all_finals.csv', index = False)

## Get the list of what the order and placement was for all finals participants for all years

In [8]:
import requests
import pandas as pd
from io import StringIO as SIO

is_refresh = False

if is_refresh == True:
    # set base url
    url = 'https://eschome.net/databaseoutput401.php'

    # use list of years from previous step
    
    # loop through all years to get list of results plus order of performance
    for index, row in all_finals.iterrows():
        print("Importing results for " + str(row['Year']) + "...")
        # set year
        if(row['Year'] < 2004):
            jahr = str(row['Year'])
        else:
            jahr = str(row['Year'])+"F"

        # get all the pages and append to all_votes
        params = {'jahr' : jahr, 'x' : '6', 'y' : '6'}
        page = requests.post(url,data = params,allow_redirects=False)

        # create a list of tables
        list_of_tables = pd.read_html(SIO(page.text), header = 0)
        if(len(list_of_tables) < 2):
            print("error - no tables found on this page.")
            continue

        df = list_of_tables[1][['Place','Points','No.','Country']].copy()
        df['Year'] = str(row['Year'])
        df = df[['Year','Place','Points','No.','Country']]
        df.rename(columns={'No.': 'Order'}, inplace=True)
                
        if(index == 0):
            all_placements = df.copy()
        else:
            all_placements = pd.concat([all_placements, df], ignore_index=True)
        
        print("Running total of " + str(len(all_placements)) + " years of finals placement records imported.")
        print(all_placements.tail(1))
else:
    all_placements = pd.read_csv('all_placements.csv')
    print("Imported " + str(len(all_placements)) + " years of finals placement records.")

#convert all numeric columns to integer and save to csv
all_placements['Place'] = all_placements['Place'].fillna(0)
all_placements['Points'] = all_placements['Points'].fillna(0)
all_placements['Order'] = all_placements['Order'].fillna(0)
all_placements = all_placements.astype({'Year': 'int32', 'Place': 'int32', 'Points': 'int32', 'Order': 'int32'})

# assign max order to each row in all_placements
all_placements['MaxOrder'] = all_placements.groupby('Year')['Order'].transform('max')
all_placements['AdjOrder'] = round((all_placements['MaxOrder'] * all_placements['Order'])/26)
all_placements.to_csv('all_placements.csv', index = False)

Imported 1399 years of finals placement records.


## Get the list of who voted for whom over time and with how many points

In [15]:
import requests
import pandas as pd
from io import StringIO as SIO

if is_refresh == True:

    # set base url
    url = 'https://eschome.net/databaseoutput403.php'

    # get list of countries
    # note:  country list was pulled from source code of https://eschome.net/index.html and massaged in excel
    countries = pd.read_csv('all_countries.csv')

    # loop through all countries to get all votes from all other countries
    for reciever_index, receiver_row in countries.iterrows():
        for giver_index, giver_row in countries.iterrows():
            
            # countries are not allowed to vote for themselves.
            if( reciever_index == giver_index ):
                continue

            else:
                print("Importing ratings for " + receiver_row['Name'] + " from " + giver_row['Name'] + "...")

                # get all the pages and append to all_votes
                params = {'land_erhalten' : receiver_row['Code'], 'land_gegeben' : giver_row['Code'], 'x' : '7', 'y' : '3'}
                page = requests.post(url,data = params,allow_redirects=False)

                # create a list of tables
                list_of_tables = pd.read_html(SIO(page.text), header = 0)
                if(len(list_of_tables) < 2):
                    print("error - no tables found on this page.")
                    continue

                # create dataframe if it doesn't exist yet, otherwise append
                df = list_of_tables[1][['Year','Type','Points']].copy()
                df['Receiver'] = receiver_row['Code']
                df['Giver'] = giver_row['Code']
                df = df[['Giver','Receiver','Year','Type','Points']]
                        
                if(reciever_index == 0 and giver_index == 1):
                    all_votes = df.copy()
                else:
                    all_votes = pd.concat([all_votes, df], ignore_index=True)
                
                print("Running total of " + str(len(all_votes)) + " vote records imported.")
                print(all_votes)


else:
    all_votes = pd.read_csv('all_votes.csv')
    print("Imported " + str(len(all_votes)) + " years of finals placement records.")

#convert all numeric columns to integer and save to csv
all_votes['Year'] = all_votes['Year'].fillna(0)
all_votes['Points'] = all_votes['Points'].fillna(0)
all_votes['Type'] = all_votes['Type'].fillna('F')
all_votes = all_votes.astype({'Year': 'int32', 'Points': 'int32'})
#save to csv
all_votes.to_csv('all_votes.csv', index = False)


Imported 49162 years of finals placement records.
